## Temperature forecasting for different cities in the world

## Introduction
For this project you are asked to analyze three datasets, called respectively:
1. pollution_us_2000_2016.csv
2. greenhouse_gas_inventory_data_data.csv
3. GlobalLandTemperaturesByCity.csv

You are asked to extract from dataset 2 only the US countries (for which we have info in the other datasets) and to perform the following tasks:
- to measure how pollution and temperature create cluster tracing the high populated cities in the world
- to analyze the correlation between pollution data and temperature change.
- to predict the yearly temperature change of a given city over a given time period, using the <b>ARIMA model</b> for <b>time series forecasting</b>, that is a model for time series forecasting integrating AR models with Moving Average.
- (OPTIONAL) rank the 5 cities that will have a highest temperature change in US


### TASK1 :Cluster Analysis
You use K-means or DBSCAN to perform the cluster analysis, and create a new dataset where the cities are associated to the different identified clusters

### TASK 2: Correlation Analysis

You measure the correlation between:
- temperature and latitude
- temperature and pollution
- temperature change (difference between the average temperature measured over the last 3 years and the previous temperature) and pollution


### TASK 3: Predicting the Temperature of a Given City across a Specified Time Period
After reading the data in the temperature data set, for each city cluster, before applying the ARIMA model you perform the following steps:

- EDA
- data cleaning and preprocessing (Converting the 'dt' (date) column to DateTime format, removing NaN)
- feature selection
- make the time-series stationary
- check for stationarity : Calculating the Augmented Dickey-Fuller Test statistic 
- identify the (p, q) order of the ARIMA model using ACF partial autocorrelation plot

Then:

-fit the ARIMA model using the calculated p, q values.
-calculate the MSE with respect to the true temp. measurements to estimate the performance of the model


NOTE: ARIMA models need the data to be stationary i.e. the data must not exhibit trend and/or seasonality. To identify and remove trend and seasonality, we can use
- seasonal decomposition
- differencing

In [3]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA, ARMAResults
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets
import seaborn as sns
from nfunctions import * 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## SECTION 1: Cluster Analysis

In [2]:
# # read the csv file containing the polluters 
# df_pollution = load_data("Project_3/data/data-project3/pollution_us_2000_2016.csv")
# df_temp = load_data("Project_3/data/data-project3/GlobalLandTemperaturesByCity.csv")
# df_ghg = load_data("Project_3/data/data-project3/greenhouse_gas_inventory_data_data(1).csv")

In [4]:
import random
from datetime import datetime, timedelta

# List of U.S. states
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
    "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey",
    "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
    "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

# Function to generate random dates within a range
def random_date(start, end):
    delta = end - start
    random_days = random.randint(0, delta.days)
    return start + timedelta(days=random_days)

# Define date range
start_date = datetime(2020, 1, 1)
end_date = datetime(2025, 1, 1)

# Generate data
num_samples = 1000
data = {
    "City": [random.choice(states) for _ in range(num_samples)],
    "Latitude": [round(random.uniform(24.396308, 49.384358), 6) for _ in range(num_samples)],  # USA Latitude range
    "Longitude": [round(random.uniform(-125.000000, -66.934570), 6) for _ in range(num_samples)],  # USA Longitude range
    "Pollution": [random.randint(10, 500) for _ in range(num_samples)],  # Random pollution level
    "CO2": [random.randint(300, 500) for _ in range(num_samples)],  # CO2 levels in ppm
    "Date": [random_date(start_date, end_date).strftime('%Y-%m-%d') for _ in range(num_samples)],
}

# Create DataFrame
df = pd.DataFrame(data)

# Display first few rows
print(df.head())

       City   Latitude   Longitude  Pollution  CO2        Date
0    Hawaii  43.277245 -102.871518        199  321  2024-12-12
1  Delaware  39.551978  -97.648389        275  314  2024-09-03
2   Florida  45.314368  -94.745129        411  326  2024-03-04
3   Florida  29.682770 -112.736343         88  477  2024-11-30
4   Alabama  27.350475  -88.924688        161  456  2023-04-19


## SECTION 2: Correlation Analysis

##SECTION 3: ARIMA model for temperature forecasting

In [ ]:
# Although we can determine p, q values manually by looking at the ACF and PACF plots for a given city, we must automate the process
#(OPTIONAL) To automate the process, we must perform a grid search over different values of p and q and choose the ARIMA model for which the AIC and BIC values are minimum

p_range = q_range = list(range(0,#))  # taking values from 0 to # (decide this looking at PACF)

aic_values = []
bic_values = []
pq_values = []

for p in p_range:
    for q in q_range:
        try:
            model = ARIMA(city_df, order=(p, d, q))
            results = model.fit(disp=-1)
            aic_values.append(ARMAResults.aic(results))
            bic_values.append(ARMAResults.bic(results))
            pq_values.append((p, q))
        except:
            pass

best_pq = pq_values[aic_values.index(min(aic_values))]  # (p,q) corresponding to lowest AIC score
print("(p,q) corresponding to lowest AIC score: ", best_pq)

In [ ]:
# fitting an ARIMA model with chosen p, d, q values and calculating the mean squared error
from sklearn.metrics import mean_absolute_error

arima_model = ARIMA(city_df, order=(best_pq[0], 0, best_pq[1])).fit()
predictions = arima_model.predict(start=0, end=len(city_df)-1)



## Conclusion

write here the report for the project